In [1]:
import sys
import numpy as np
import pickle as pkl
from IPython.display import display
from maraboupy import Marabou, MarabouCore
from utils import TOTUtils

In [2]:
model_path = '../artifacts/models/model_v3.2.0/model.h5'
data_csv = '../data/v3.2.0/test.csv'
all_samples = TOTUtils.load_samples(data_csv, frac=0.001)
samples = TOTUtils.filter_samples(all_samples, model_path)
print(f'{len(samples)} correct samples')

135 correct samples


In [3]:
# from sensitivity import find_feature_sensitivity_boundaries

# find_feature_sensitivity_boundaries('../artifacts/models/model_v3.2.0/model.h5', 0, samples[::], d_min=0.00001, d_max=50, multithread=True, verbose=1)

In [7]:
from sensitivity import find_sensitivity_boundaries

results = find_sensitivity_boundaries(model_path, samples[:10], d_min=0.00001, d_max=100, multithread=True, save_results=True, save_samples=True)

for x,result in results.items():
    dists,_ = result
    print(x, dists)

INFO:sensitivity:wrote summary to ../logs/sensitivity/summary.csv
INFO:sensitivity:wrote detils to ../logs/sensitivity/details.csv
x0 (-1.7407000000000012, 0.49878000000000017)
x1 (-0.008790000000000058, 0.36157000000000017)
x2 (-0.06451000000000004, 0.0452900000000001)
x3 (-1.7451300000000003, 1.33788)
x4 (-0.80492, 0.3280500000000002)
x5 (-0.8274100000000001, 0.0892800000000003)
x6 (-0.4095800000000001, 0.09878000000000006)
x7 (-1.1085399999999994, 0.16702000000000006)
x8 (-0.1643000000000001, 0.4914700000000002)
x9 (-0.47933000000000014, 0.3772600000000002)
x10 (-0.16600000000000004, 0.46379000000000015)
x11 (-0.056590000000000105, 0.18799)
x12 (-0.9590799999999997, 0.6053499999999999)
x13 (-0.09950000000000003, 0.7420999999999998)
x14 (-0.36130000000000023, 1.35757)
x15 (-0.4227800000000001, 2.2329300000000054)
x16 (-0.5460100000000001, 0.12464000000000007)
x17 (-0.14683999999999997, 0.14640999999999998)
x18 (-0.02320000000000006, 0.16060999999999998)
x19 (-0.008010000000000066, 0.

In [5]:
# from tensorflow.keras.models import load_model

# model = load_model('../artifacts/models/model_v3.2.0/model.h5')
# inputs,outputs = samples[6]
# d = 49.69331
# x = 0
# pred = model.predict([inputs[0:x] + [inputs[x]+d] + inputs[x+1:]])

# # assert that prediction was correct
# assert(outputs.index(max(outputs)) == list(pred[0]).index(max(pred[0])))

In [6]:
# from maraboupy import Marabou, MarabouCore

# pb_path = '../artifacts/models/model-v3.1.4/model.pb'
# net = Marabou.read_tf(pb_path)

# sample = samples[0]
# inputs, outputs = sample
# expected_cat = outputs.index(max(outputs))
# other_cats = [i for i in range(len(outputs)) if i != expected_cat]

# for x,v in enumerate(inputs):
#     net.setLowerBound(net.inputVars[0][x], v)
#     net.setUpperBound(net.inputVars[0][x], v)
# for c in other_cats:
#     print('OC:', c)
#     eq1 = MarabouCore.Equation(MarabouCore.Equation.LE)
#     eq1.addAddend(1, net.outputVars[0][c])
#     eq1.addAddend(-1, net.outputVars[0][0])
#     eq1.setScalar(0)
# pred = net.evaluate([input_sample], useMarabou=False)
# print(pred)